In [ ]:
import pandas as pd
import numpy as np
# wizualizacje
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.linear_model import PoissonRegressor, GammaRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, r2_score #, d2_absolute_error_score
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, mutual_info_regression, mutual_info_classif, f_regression, RFECV, SequentialFeatureSelector

from patsy import dmatrices
import statsmodels.api as sm

# ustawienie szerokiego ekranu wyświetlacza JNotebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# opcje wyświetlania w Pandas
# - maks. 55 kolumn
# - maks. 101 wierszy
# - liczby w notacji dziesiętnej z czterema zerami po przecinku
pd.set_option('display.max_columns', 55)
pd.set_option('display.max_rows', 101)
pd.set_option('display.float_format', lambda x: f"{x:.4f}")

# opcje formatowania wykresów matplotlib
# - etykiety osi: bold
# - tekst: bold
# - domyślny rozmiar fontu=14
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['font.size'] = '14'

# Wczytanie danych

In [ ]:
# Zadanie 1

filepath = r"C:\Users\pnaumczyk\Documents\Dane\Python_modelowanie_crashCourse\train.csv"
cats_dow = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Weekend"]
dow_dtype = pd.api.types.CategoricalDtype(categories=cats_dow, ordered=True)

miesiac_zima = ["January", "February", "December"]
miesiac_wiosna = ["March", "April", "May"]
miesiac_lato = ["June", "July", "August"]
miesiac_jesien = ["September", "October", "November"]

cats_season = ["wiosna", "lato", "jesien", "zima"]
season_dtype =  pd.api.types.CategoricalDtype(categories=cats_season, ordered=True)

cat_cols = [
    'pora_dnia_przejazdu',
    'dzien_tygodnia_cat', 
    'stacja_start_cat',  
    'stacja_finisz_cat', 
    'rodzaj_pociagu_cat', 
    'pora_roku_cat'
]
num_cols = [
    'opoznienie_start',
    'czy_szkoda',
    'wyplata_szkoda'
]


df = (
    pd.read_csv(filepath, sep=';')
    
    .assign(
        **{
             k: lambda df_, col = k: pd.to_datetime(df_[col])
                for k in [
                    'Kiedy jechał*ś?'
                ]
        },
        **{ k: lambda df_, col = k: pd.to_numeric(df_[col], downcast='float')
                for k in [
                    'Koszt podróży:'
                ]
        },
        **{ k: lambda df_, col = k: pd.to_numeric(df_[col], downcast='integer')
                for k in [
                    'Liczba minut opóźnienia na starcie:', 'Liczba minut opóźnienia na mecie:'
                ]
        }
    )
    
    .astype({
        **{ k: 'string' 
               for k in [
                   'Stacja początkowa:', 'Stacja końcowa:', 'Jak wrażenia z podróży? :D (nieobowiązkowe)', 'Uwagi (nieobowiązkowe):'
               ]            
        },
        **{ k: 'category' 
               for k in [
                   'Rodzaj pociągu:', 'Pora dnia:'
               ]            
        }
    })
    
    .rename(columns={
        'Kiedy jechał*ś?'         : 'data_przejazdu',
        'Stacja początkowa:'      : 'stacja_start',
        'Stacja końcowa:'         : 'stacja_finisz',
        'Rodzaj pociągu:'         : 'rodzaj_pociagu',
        'Uwagi (nieobowiązkowe):' : 'uwagi',
        'Koszt podróży:'          : 'cena_biletu',
        'Pora dnia:'              : 'pora_dnia_przejazdu',
        'Liczba minut opóźnienia na starcie:' : 'opoznienie_start',
        'Liczba minut opóźnienia na mecie:'   : 'opoznienie_finisz',
        'Jak wrażenia z podróży? :D (nieobowiązkowe)' : 'wrazenia'
    })
    
    .drop(columns=['wrazenia', 'uwagi'])
    .drop(index=[30, 50])
    
    .assign(
        pora_dnia_przejazdu = lambda df_: np.select(
        [
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_start.str.contains('Legionowo')).astype(bool),
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_finisz.str.contains('Legionowo')).astype(bool),
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_finisz.str.contains('Gdask')).astype(bool)
        ],
        [
            'rano',
            'popołudnie',
            'popołudnie'            
        ],
        df_.pora_dnia_przejazdu
    ))
    
    .reset_index(drop=True)
    
    .assign(
        czy_szkoda = lambda df_: np.select(
            [
                df_.opoznienie_finisz.gt(10) & df_.opoznienie_finisz.le(40),
                df_.opoznienie_finisz.gt(40)
            ],
            [
                1,
                2
            ],
            0
        ),
        
        wyplata_szkoda = lambda df_: np.where(
            df_.opoznienie_finisz.gt(10),
            df_.cena_biletu * np.exp(df_.opoznienie_finisz.div(100)) * df_.czy_szkoda,
            0.0
        )

    )
    
    .sort_values(by='data_przejazdu')
    .reset_index(drop=True)
    
    .assign(
        dzien_tygodnia = lambda df_: df_.data_przejazdu.dt.day_name().astype('category'),
        dzien_tygodnia_cat = lambda df_: np.where(
            df_.dzien_tygodnia.eq('Sunday') | df_.dzien_tygodnia.eq('Saturday') | df_.dzien_tygodnia.eq('Monday'),
            'Weekend',
            df_.dzien_tygodnia
        ),
        miesiac = lambda df_: df_.data_przejazdu.dt.month_name().astype('category'),
        pora_roku = lambda df_: np.select(
            [
                df_.miesiac.isin(miesiac_zima),
                df_.miesiac.isin(miesiac_wiosna),
                df_.miesiac.isin(miesiac_lato),
                df_.miesiac.isin(miesiac_jesien)
            ],
            [
                "zima",
                "wiosna",
                "lato",
                "jesien"
            ],
            "brak_danych"
        ),        
        stacja_start_cat = lambda df_: np.select(
            [
                df_.stacja_start.str.contains("Warszawa").astype(bool),
                df_.stacja_start.str.contains("Legionowo").astype(bool),
                (df_.stacja_start.str.contains("Gdańsk") | df_.stacja_start.str.contains("Gdask")).astype(bool)
            ],
            [
                "Warszawa",
                "Legionowo",
                "Gdańsk"
                
            ],
            "Inne"
        ),
        stacja_finisz_cat = lambda df_: np.select(
            [
                df_.stacja_finisz.str.contains("Warszawa").astype(bool),
                df_.stacja_finisz.str.contains("Legionowo").astype(bool),
                (df_.stacja_finisz.str.contains("Gdańsk") | df_.stacja_finisz.str.contains("Gdask")).astype(bool)
            ],
            [
                "Warszawa",
                "Legionowo",
                "Gdańsk"
                
            ],
            "Inne"
        ),  
        rodzaj_pociagu_cat = lambda df_: np.where(df_.rodzaj_pociagu.eq("EIC"), '"Zwykły" InterCity', df_.rodzaj_pociagu)
    )
    
    .astype({
        **{ k: 'category' 
               for k in [
                   'stacja_start_cat', 'stacja_finisz_cat', 'rodzaj_pociagu_cat'
               ]            
        },      
    })
    
    .assign(
        dzien_tygodnia_cat = lambda df_: df_.dzien_tygodnia_cat.astype(dow_dtype),
        pora_roku_cat = lambda df_: df_.pora_roku.astype(season_dtype)
    )
    
    .drop(columns=[
        'data_przejazdu', 'stacja_start', 'stacja_finisz', 'rodzaj_pociagu', 'cena_biletu', 'dzien_tygodnia', 'miesiac', 'pora_roku', 'opoznienie_finisz'
    ])

    .reindex(columns = cat_cols + num_cols)

)
df

# Preprocessing (podział na set treningowy i testowy)

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.3, stratify=df.czy_szkoda, random_state=42)
print(
    f"Rozmiar testowego: {df_test.shape}\n"
    f"Rozmiar treningowego: {df_train.shape}"
)

# Spis treści
1. [Automatyczne przekształcenia zmiennych](#column_transformer)
2. [Walidacja krzyżowa](#cv)
3. [Selekcja zmiennych wyjaśniających](#fs)
4. [Ścieżki przekształceń](#pipeline)

***
***

# <a id='column_transformer'>Automatyczne przekształcenia zmiennych</a> 

1. Stwórz transformer column_trans przekształcający dane wejściowe: OneHotEncoding dla zmiennych kategorialnych, MinMaxScaler dla ciagłych, zmienne wyjaśniane bez przekształceń <br>
Wyświetl jego zawartość

2. Dopasuj transformer na danych treningowych tworząc zmienną X_train_coded. Co jest efektem przekształcenia?

3. Wyświetl parametry poszczególnych przekształceń w column_trans

4. Wyświetl nazwy poszczególnych kolumn po przekształceniu

5. Zastosuj przekształcenie kolumn osobno na danych testowych i treningowych tworząc zmienne X_train_coded oraz X_test_coded. Jaki jest ich format?

6. Stwórz zmienne df_train_tans oraz df_test_trans typu pd.DataFrame, przechowujące przekształcone dane

7. Usuń z podanych wzorów kolumny ze zmiennymi wyjaśnianymi

# <a id='cv'>Walidacja krzyżowa</a> 

1. Zdefiniuj zmienną params jako parametry regresji Poissona wyszukując wśród wartości:
    1. parametr alpha: 100 wartości logarytmicznie rozkładających sie od 10^-20 do 10
    2. parametr solver: w zależności od wersji scikit-learn: albo wyłącznie 'lbfgs', albo którekolwiek spośród: {'lbfgs', 'newton-cholesky'}
    3. parametr max_iter: 100 lub 1000

2. Stwórz zmienną poisson_gscv zawierającą wyniki GridSearchCV na zbiorze df_train_trans dla zmiennej wyjaśnianej czy_szkoda:
    1. model - regresja Poissona
    2. parametry - z zadania 1
    3. scoring - mean_poisson_deviance
    4. typ walidacji krzyżowej - stratified k fold z 10 foldami


3. Wyświetl wyniki walidacji krzyżowej

4. Wyświetl najlepsze wyniki

5. Zrób grid search uwzględniając trzy metryki: 
    1. mean_poisson_deviance
    2. r2_score
    3. (jeśli masz scikit-learn powyżej 1.1) d2_absolute_error_score
    
Wyświetl najlepsze parametry dopasowania dla każdej z metryk

6. Przeprowadź grid search jak w punkcie 5, ale ze StratifiedShuffleSplit

7. Policz metryki na zbiorze testowym z dopasowaniem domyślnym oraz z dopasowaniem alpha według grid search. Porównaj.

# <a id='fs'>Selekcja zmiennych wyjaśniających</a> 

1. Przelicz i wyświetl wartości informacji wzajemnej dla zmiennej czy_szkoda z użyciem SelectKBest

2. Jakie cztery zmienne maja największą informację wzajemną ze zmienną czy_szkoda? Jakie mają największy związek liniowy?

2. A. (dodatkowe) Ustaw w informacji wzajemnej - które zmienne są kategorialne

3. Przeprowadź dopasowanie RFECV dla zmiennej czy_szkoda i 10 foldów cv

4. Wyświetl wyniki dopasowania RFECV w postaci data frame

5. Jakie zmienne pozostały po selekcji RFECV?

6. Powtórz dopasowanie RFECV tym razem ustalając parametr alfa jako jeden z tych z grid search. Czy to zmieniło listę zmiennych?

7. Przeprowadź selekcję wprzód z użyciem SequentialFeatureSelector. Jakie zmienne zostały w modelu?

8. Powtórz punkt 7 z parametrem alpha wyliczonym z grid search. Czy zmieniło to wyniki?

# <a id='pipeline'>Ścieżki przekształceń</a> 

1. Stwórz pipeline zawierający przekształcenie kolumn zdefiniowane wcześniej oraz modelowanie PoissonRegressor. Wyświetl pipeline

2. Wyświetl parametry pipeline'u

3. Przeprowadź optymalizację parametrów PoissonRegressor przy użyciu GridSearchCV, ale bez "information leakege" podczas walidacji krzyżowej (wykorzystaj pipeline). 

4. Porównaj uzyskane parametry oraz metryki oceniające model z wcześniej uzyskanymi

5. Stwórz nowy pipeline z dwiema metodami skalowania dla zmiennych numerycznych (StandardScaler orpócz MinMaxScaler). Porównaj, który daje najlepsze wyniki pod względem r^2 oraz neg_mean_poisson_deviance

6. Połącz powyższe i przelicz optymalne parametry wraz z wyborem ścieżki przetwarzania

7. Porównaj metryki uzyskane na przewidywaniu zbioru testowego na podstawie danych z punktu 6 i wcześniejszych